# Knapsack Problem (KP)

Given a set of $n$ items with profit $p_i$ and weight $w_i$ for item $i=0,\ldots,n-1$, and the capacity value $c$ (all parameters are positive integers), select a subset of the items such that the total profit of the selected items is maximized and the total weight does not exceed $c$.

Example applications:
* <b>Mountaineer</b>: A mountaineer is packing his knapsack for a mountain tour and has to decide which items he should take with him. He has a large number of objects available which may be useful on his tour. Each of these items would give him a certain amount of comfort or benefit $p_i$. However, the weight $w_i$ of every object which the mountaineer puts into his knapsack increases the load he has to carry. For obvious reasons, he wants to limit the total weight of his knapsack and hence fixes the maximum load by the capacity value $c$.
* <b>Investment</b>: A wealthy individual or institutional investor has a certain amount of money $c$ available which she wants to put into profitable business projects. As a basis for her decisions she compiles a long list of possible investments including for every investment the required amount $w_i$ and the expected net return $p_i$ over a fixed period. The aspect of risk is not explicitly taken into account here. Obviously, the combination of the binary decisions for every investment such that the overall return on investment is as large as possible can be formulated as a knapsack problem.
* <b>Airline cargo</b>: The dispatcher of a cargo airline has to decide which of the transportation requests posed by the customers he should fulfill, i.e. how to load a particular plane. His decision is based on a list of requests which contain the weight $w_i$ of every package and the rate per weight unit charged for each request. Note that this rate is not fixed but depends on the particular longterm arrangements with every customer. Hence the profit $p_i$ made by the company by accepting a request and by putting the corresponding package on the plane is not directly proportional to the weight of the package. Naturally, every plane has a specified maximum capacity $c$ which may not be exceeded by the total weight of the selected packages.

# Mathematical Model for KP

Variables: $x_i=1$ if item $i$ is selected, $=0$ otherwise, for $i=0,\ldots,n-1$

$\max \sum_{i=0}^{n-1} p_i x_i$ (total profit)

subject to:

$\sum_{i=0}^{n-1} w_i x_i \leq c$ (total weight)

$x_i\in\{0,1\}$ for $i=0,\ldots,n-1$

In [6]:
import pandas as pd
import numpy as np
from gurobipy import *

c = 9                     # knapsack capacity
p = [6, 5, 8, 9, 6, 7, 3] # item profits
w = [2, 3, 6, 7, 5, 9, 4] # item weights

In [23]:
def model_kp(c, p, w, LogToConsole=True, TimeLimit=60):
    n = len(p)
    assert n == len(w), 'Lengths of p and w should be the same'
    model = Model()
    model.params.LogToConsole = LogToConsole
    model.params.TimeLimit = TimeLimit # seconds
    x = model.addVars(n, vtype=GRB.BINARY)
    model.setObjective(quicksum(p[i] * x[i] for i in range(n)), GRB.MAXIMIZE)
    model.addConstr(quicksum(w[i] * x[i] for i in range(n)) <= c)
    model.optimize()
    items_selected = [i for i in range(n) if x[i].X > 0.5]
    total_profit = int(model.ObjVal)
    return items_selected, total_profit

In [24]:
items_selected, total_profit = model_kp(c, p, w, False)
print('Items selected are', items_selected, 'and the total profit is', total_profit)

Items selected are [0, 3] and the total profit is 15


In [36]:
from random import randint, seed
seed(124)
c = 1000
w = [randint(5, 10) * 21 for i in range(100)]
p = [randint(100, 120) * w[i] for i in range(100)]

items_selected, total_profit = model_kp(c, p, w, False)
print('Items selected are', items_selected, 'and the total profit is', total_profit)

Items selected are [11, 23, 62, 73, 81, 99] and the total profit is 118083


In [3]:
def model_gs_matching(n_b, n_l, u, c, q, lambda_1, lamda_2, LogToConsole=True, TimeLimit=60):
    model = Model()
    model.params.LogToConsole = LogToConsole
    model.params.TimeLimit = TimeLimit # seconds
    x = {}
    for b_idx in range(n_b):
        x[b_idx] = {}
        for l_idx in range(n_l): 
            s = "x_{}_{}".format(b_idx, l_idx)
            x[b_idx][l_idx] = model.addVars(s, vtype=GRB.BINARY, name=s)
            w[b_idx][l_idx] = model.addVars(s, vtype.GRB.BINARY, name=s)

    for l_idx in range(n_l):
        model.addConstr(quicksum(x[b_idx][l_idx] for b_idx in range(n_b)) <= 1)
    
    for b_idx in range(n_b):
        model.addConstr(quicksum((q[l_idx]*x[b_idx][l_idx]) for l_idx in range(n_l)) >= 1)
    
    for b_ix in range(n_b):
        for l_idx in range(n_l):
            constr_obj_1 = c[b_idx]*x[b_idx][l_idx]
            constr_obj_2 = 0
            constr_obj_3 = 0
            
            for b_idx_2 in range(n_b):
                if b_idx != b_idx_2:
                    if u[b_idx][l_idx] < u[b_idx_2][l_idx]:
                        constr_obj_2 += (x[b_idx][l_idx])
            constr_obj_2 *= c[b_idx] 

            for l_idx_2 in range(n_l):
                if l_idx != l_idx_2:
                    if u[b_idx][l_idx] < u[b_idx][l_idx_2]:
                        constr_obj_3 += (q[l_idx] * x[b_idx][l_idx])
                        
            model.addConstr((constr_obj_1 + constr_obj_2 + constr_obj_3) >= (c[b_idx]*w[b_idx][l_idx]))
                    
    model.setObjective(quicksum(p[i] * x[i] for i in range(n)), GRB.MAXIMIZE)
#     model.optimize()
#     items_selected = [i for i in range(n) if x[i].X > 0.5]
#     total_profit = int(model.ObjVal)
#     return items_selected, total_profit

SyntaxError: invalid syntax (<ipython-input-3-87a0df049705>, line 17)